# Study of Gradient Descent with squared error in Python
This notebook is a study of gradient descent technique in Python using the saqured error function.

<br>

Formulas reference:

<br> 

- Weight update value
$$ \Delta w_i = \alpha \delta x_i $$ 

<br>

- Error term
$$ \delta  = (y-\hat{y}){f}'(h) = (y-\hat{y}){f}'(\sum w_i x_i) $$
where : $(y-\hat{y})$ is the output error, and ${f}'(h)$ is the derivative of the activation function, $f(h)$.

### Imports

In [2]:
import numpy as np

### Simple Python implementation
Only one input unit, using sigmoid activation function $f(h)$.

In [4]:
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# Derivative of the sigmoid activation function
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Simulation of gradient descent

learning_rate = 0.5
x = np.array([1, 2, 3, 4])
y = np.array(0.5)
w = np.array([0.5, -0.5, 0.3, 0.1])

# Prediction or feedforward
h = np.dot(x, w)

# NN output
output = sigmoid(h)

# Error calculation
error = (y - output)

# Error term
error_term = error * sigmoid_prime(h)

# Weights update
del_w = [learning_rate * error_term * x]

print('Neural Network output:')
print(output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

Neural Network output:
0.6899744811276125
Amount of Error:
-0.1899744811276125
Change in Weights:
[array([-0.02031869, -0.04063738, -0.06095608, -0.08127477])]
